# Tutorial - Stable Baselines

## Índice

- [O que é Stable Baselines?](#O-que-é-Stable-Baselines?)
- [Como usar Stable Baselines?](#Como-usar-Stable-Baselines?)
  - [Importando o Gym](#Importando-o-Gym)
  - [O que é um Ambiente?](#O-que-é-um-Ambiente?)
  - [Como Funciona um Ambiente do Gym?](#Como-Funciona-um-Ambiente-do-Gym?)
  - [Criando um Ambiente](#Criando-um-Ambiente)
  - [Criando um Agente](#Criando-um-Agente)
  - [Rodando um Episódio](#Rodando-um-Episódio)
  - [Treinamento](#Treinamento)


## O que é Stable Baselines?

A **[Stable Baselines](https://github.com/hill-a/stable-baselines)** é uma biblioteca de Aprendizado por Reforço que implementa diversos algoritmos de agentes de RL, além de várias funcionalidades úteis para o treinamento de um agente. Suas implementações são bem simples e intuitivas, mas sem deixarem de ser otimizadas e poderosas, buscando facilitar o desenvolvimento de projetos de reforço de alta qualidade.

![Logo](https://github.com/hill-a/stable-baselines/raw/master/docs//_static/img/logo.png "Logo da Stable Baselines")

## Como usar Stable Baselines?

Com Stable Baselines, o processo de criar e treinar um agente é bem simples. Entretanto, caso você não saiba muito de Aprendizado por Reforço, é primeiro preciso passar por alguns conhecimentos básicos.

### Importando o Gym

O **[Gym](https://gym.openai.com/)** é uma biblioteca desenvolvida pela OpenAI que contém várias implementações prontas de ambientes de Aprendizado por Reforço. Ela é muito utilizada quando se quer testar um algoritmo de agente sem ter o trabalho de programar seu próprio ambiente.

<img src="https://user-images.githubusercontent.com/10624937/42135602-b0335606-7d12-11e8-8689-dd1cf9fa11a9.gif" alt="Exemplos de Ambientes do Gym" class="inline"/>
<figcaption>Exemplo de Ambientes do Gym</figcaption>
<br>

Para se ter acesso a esses ambientes, basta importar o Gym da seguinte forma:

In [1]:
import gym

### Como Funciona um Ambiente do Gym?

Agora que você já sabe o que é um ambiente, é preciso entender como nosso agente interage efetivamente com ele. Todos os ambientes do Gym possuem alguns métodos simples para facilitar a comunicação com eles:

<br>

| Método               | Funcionalidade                                          |
| :------------------- |:------------------------------------------------------- |
| reset()              | Inicializa o ambiente e recebe a observação inicial     |
| step(action)         | Executa uma ação e recebe a observação e a recompensa   |
| render()             | Renderiza o ambiente                                    |
| close()              | Fecha o ambiente                                        |

<br>

Assim, o código para interagir com o ambiente costuma seguir o seguinte modelo:

---

```python
ambiente = gym.make("Nome do Ambiente")                         # Cria o ambiente
observação = ambiente.reset()                                   # Inicializa o ambiente
acabou = False

while not acabou:
    ambiente.render()                                           # Renderiza o ambiente
    observação, recompensa, acabou, info = ambiente.step(ação)  # Executa uma ação
    
ambiente.close()                                                # Fecha o ambiente
```

---

### Criando um Ambiente

Para utilizar um dos ambientes do Gym, nós utilizamos a função ```gym.make()```, passando o nome do ambiente desejado como parâmetro e guardando seu valor retornado em uma variável que chamaramos de ```env```. A lista com todos os ambiente pode ser encontrada [aqui](https://gym.openai.com/envs/#classic_control).

In [2]:
env = gym.make("CartPole-v1")
env.seed(0)

[0]

Nesse caso, nós vamos utilizar o ambiente ```CartPole-v1```, um ambiente bem simples que modela um pêndulo invertido em cima de um carrinho buscando seu estado de equilíbrio.

<img src="https://miro.medium.com/max/1200/1*jLj9SYWI7e6RElIsI3DFjg.gif" width="400px" alt="Ambiente do CartPole-v1" class="inline"/>

#### CartPole

Antes de treinar qualquer agente, primeiro é preciso entender melhor quais as características do nosso ambiente.

O **Espaço de Observação** do CartPole é definido por 4 informações:

<br>

|     | Informação                         | Min     | Max    |
| :-- | :--------------------------------- | :-----: | :----: |
| 0   | Posição do Carrinho                | -4.8    | 4.8    |
| 1   | Velocidade do Carrinho             | -Inf    | Inf    |
| 2   | Ângulo da Barra                    | -24 deg | 24 deg |
| 3   | Velocidade na Extremidade da Barra | -Inf    | Inf    |

<br>

Dessa forma, a cada instante recebemos uma lista da observação com o seguinte formato:

In [3]:
print(env.observation_space.sample())

[ 3.3661270e+00 -1.4129330e+38  2.4853730e-01  1.2598591e+38]


Já o **Espaço de Ação** é composto por duas ações únicas: mover o carrinho para a **esquerda** ou para a **direita**.

Quando queremos mover o carrinho para a esquerda, fazemos um `env.step(0)`; quando queremos movê-lo para a direita, enviamos um `env.step(1)`

In [4]:
print(env.action_space.sample())

0


### Criando um Agente

Depois de escolhermos nosso ambiente, já podemos pensar em qual algoritmo de agente queremos usar.

A biblioteca disponibiliza algoritmos de diversos tipos, como *Deep Q-Networks* e *Actor-Critics*. Nem todos eles suportam todos os tipos de ambientes, então é recomendável dar uma olhada na [página oficial dos algoritmos](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html).

#### Inicialização

Todos os algoritmos são inicializados de uma forma parecida, nós instanciamos eles com alguns parâmetros em comum: ```policy```, que define a arquitetura da rede neural e ```env```, que define o ambiente no qual o agente vai treinar. Assim, a inicialização segue o seguinte formato:

```python
agente = ALGORITMO(policy, env)
```

Como exemplo, vamos criar um **PPO**, um tipo de Actor-Critic, para resolver o ambiente do **CartPole**:

In [5]:
from stable_baselines3 import PPO

model = PPO('MlpPolicy', env, seed=1, verbose=1)

Using cuda device
Wrapping the env in a DummyVecEnv.


Inicializamos o PPO com a `policy` 'MlpPolicy', que cria uma rede neural artificial com 2 camadas ocultas de 64 neurônios cada, no `env` do CartPole.

Todos os agentes também possuem alguns métodos em comuns bem importantes de se conhecer:

<br>

| Método        | Funcionalidade                          |
| :------------ |:--------------------------------------- |
| learn()       | Treina o agente                         |
| predict(obs)  | Escolhe uma ação com base na observação |
| save(caminho) | Salve o agente                          |
| load(caminho) | Carrega o agente                        |

<br>

Dessa forma, se quisermos escolher a próxima ação do nosso agente, nós rodamos:

```python
agente.predict(observação)
```

### Rodando um Episódio

Bom, agora que já temos nosso agente e nosso ambiente, já podemos rodar nosso primeiro episódio!

Para isso, vamos criar uma função `run_episode` para simplificar o processo:

In [6]:
import itertools

# A função recebe o ambiente e o agente como parâmetros
def run_episode(env, model, render=False):
    # Primeiro, inicializamos o ambiente e guardamos a observação inicial em 'obs'
    obs = env.reset()
    
    # Loop do episódio
    for t in itertools.count():
        # Nosso modelo prediz a ação 'action' a ser tomada com base na nossa observação 'obs'
        action, _states = model.predict(obs)
        
        # Tomamos a ação 'action', e recebemos uma nova observação 'obs', uma recompensa 'reward'
        # e se o episódio terminou 'done'
        obs, reward, done, info = env.step(action)
        
        # Renderiza o ambiente, caso desejado
        if render:
            env.render()
            
        # Finaliza o episódio, caso tenha terminado
        if done:
            break
    
    # Quando terminado, fechamos o ambiente
    env.close()
    
    # Imprimindo a duração do ambiente
    print("Duração do Episódio: " + str(t+1))

Pronto, já podemos rodar o nosso agente!

In [7]:
run_episode(env, model, render=True)

Duração do Episódio: 13


Provavelmente o resultado não foi tão bom assim. Isso é porque precisamos treinar nosso agente para que ele saiba as melhores ações a se tomar.

### Treinamento

O treinamento do agente acontece de maneira bem simples, basta rodar o método `.learn()` com a quantidade de instantes de tempo `total_timesteps` que desejamos treinar.

Ao longo do treinamento, nosso agente vai mostrando algumas informações importantes no ouput, como a função de custo `loss`.

In [8]:
model.learn(total_timesteps=25000)

-----------------------------
| time/              |      |
|    fps             | 863  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 568         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008619683 |
|    clip_fraction        | 0.172       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -4.02e+03   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.61        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0165     |
|    value_loss           | 55          |
-----------------------------------------
----------------------------------

------------------------------------------
| time/                   |              |
|    fps                  | 254          |
|    iterations           | 13           |
|    time_elapsed         | 104          |
|    total_timesteps      | 26624        |
| train/                  |              |
|    approx_kl            | 0.0009792595 |
|    clip_fraction        | 0.141        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.54        |
|    explained_variance   | 0.476        |
|    learning_rate        | 0.0003       |
|    loss                 | 12.7         |
|    n_updates            | 120          |
|    policy_gradient_loss | -0.00539     |
|    value_loss           | 53.1         |
------------------------------------------


Agora que já treinamos o nosso agente, podemos rodar mais um episódio para ver como ele melhorou:

In [9]:
run_episode(env, model, render=True)

Duração do Episódio: 500


Parabéns, você acabou de criar o seu primeiro agente com a Stable Baselines!

Esse conhecimento já é suficiente para aplicar em vários ambientes simples, mas ainda existem várias outras funcionalidades muito interessantes da biblioteca que valem a pena aprender.

TBD:

 - Ensinar a avaliar os modelos
   - ```mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)```
 - Ensinar a fazer plots